In [ ]:
import nibabel as nib 
import matplotlib.pyplot as plt 
import numpy as np
import os
from tqdm import tqdm
import seaborn as sns
import pandas as pd

In [ ]:
non_harmonized = "/nfs/masi/krishar1/SPIE_2025_InhaleExhaleCT/data_split/registration_ANTS_command_line/ANTS_outputs_exp_toinsp_nonharmonized"

files = sorted(os.listdir(non_harmonized))

cases_unharm = []
controls_unharm = []

cases_files_unharm = []

for file in tqdm(files):
    if file.endswith("_control"):
        print("Control:", file)
        control = nib.load(os.path.join(non_harmonized, file, "log_jacobian_det.nii.gz")).get_fdata()
        controls_unharm.append(control.flatten())
    else:
        #Add only 20 cases and keep a track of the name of the files
        print("Case:",file)
        case = nib.load(os.path.join(non_harmonized, file, "log_jacobian_det.nii.gz")).get_fdata()
        cases_unharm.append(case.flatten())

In [ ]:
#normalize the histograms 
def normalize_histogram(faltten, num_bins):
    hist, bin_edges = np.histogram(faltten, bins=num_bins)
    normalized_hist = hist/np.sum(hist)
    return normalized_hist, bin_edges

histograms_cases_unharm = []
histograms_controls_unharm = []
num_bins = 50 

for case, control in zip(cases_unharm, controls_unharm):
    normalized_hist_case_unharm, bin_edges_case_unharm = normalize_histogram(case.flatten(), num_bins)
    normalized_hist_control_unharm, bin_edges_control_unharm = normalize_histogram(control.flatten(), num_bins)
    histograms_cases_unharm.append(normalized_hist_case_unharm)
    histograms_controls_unharm.append(normalized_hist_control_unharm)

avg_hist_case_unharm = np.mean(histograms_cases_unharm, axis=0)
avg_hist_control_unharm = np.mean(histograms_controls_unharm, axis=0)

plt.figure(figsize=(10, 8), facecolor='w')
# kde for this normalized histogram
plt.bar(bin_edges_case_unharm[:-1], avg_hist_case_unharm, width=np.diff(bin_edges_case_unharm), color='blue', edgecolor='black', label='COPD_Case', alpha = 0.5)
plt.bar(bin_edges_control_unharm[:-1], avg_hist_control_unharm, width=np.diff(bin_edges_control_unharm), color='orange', edgecolor='black', label='COPD_Control', alpha = 0.5)
plt.xlabel('Log Jacobian Determinant Value', fontsize = 16)
plt.xticks(fontsize = 14)
plt.xlim(-1.0, 1.0)
plt.ylabel('Normalized Frequency', fontsize = 16)
plt.yticks(fontsize = 14)
plt.ylim(0, 0.15)
plt.title('Average Normalized Histogram of COPD Cases vs COPD Controls: Non Harmonized', fontsize = 16)
plt.legend()
plt.savefig("/nfs/masi/krishar1/SPIE_2025_InhaleExhaleCT/SPIE_paper_figures/quantitative/nonharmonized_jd.tiff", dpi = 300)
plt.show()

In [ ]:
non_harmonized = "/nfs/masi/krishar1/SPIE_2025_InhaleExhaleCT/data_split/registration_ANTS_command_line/ANTS_outputs_exptoinsp_harmonized"

files = sorted(os.listdir(non_harmonized))

cases = []
controls = []

for file in tqdm(files):
    if file.endswith("_control"):
        control = nib.load(os.path.join(non_harmonized, file, "log_jacobian_det.nii.gz")).get_fdata()
        controls.append(control.flatten())
    else:
        case = nib.load(os.path.join(non_harmonized, file, "log_jacobian_det.nii.gz")).get_fdata()
        cases.append(case.flatten())

In [ ]:
#normalize the histograms 
def normalize_histogram(faltten, num_bins):
    hist, bin_edges = np.histogram(faltten, bins=num_bins)
    normalized_hist = hist/np.sum(hist)
    return normalized_hist, bin_edges

num_bins = 50
histograms_cases = []
histograms_controls = []

for case, control in zip(cases, controls):
    normalized_hist_case, bin_edges_case = normalize_histogram(case.flatten(), num_bins)
    normalized_hist_control, bin_edges_control = normalize_histogram(control.flatten(), num_bins)
    histograms_cases.append(normalized_hist_case)
    histograms_controls.append(normalized_hist_control)

avg_hist_case = np.mean(histograms_cases, axis=0)
avg_hist_control = np.mean(histograms_controls, axis=0)

plt.figure(figsize=(10, 8))
#
plt.bar(bin_edges_case[:-1], avg_hist_case, width=np.diff(bin_edges_case), color='blue', edgecolor='black', label='COPD_Case', alpha = 0.5)
plt.bar(bin_edges_control[:-1], avg_hist_control, width=np.diff(bin_edges_control), color='orange', edgecolor='black', label='COPD_Control', alpha = 0.5)
plt.xlabel('Log Jacobian Determinant Value', fontsize = 16)
plt.xticks(fontsize = 14)
plt.xlim(-1.0, 1.0)
plt.ylim(0, 0.15)
plt.ylabel('Normalized Frequency', fontsize = 16)
plt.yticks(fontsize = 14)
plt.title('Average Normalized Histogram of COPD Cases vs COPD Controls: Harmonized', fontsize = 16)
plt.legend()
plt.savefig("/nfs/masi/krishar1/SPIE_2025_InhaleExhaleCT/SPIE_paper_figures/quantitative/harmonized_jd.tiff", dpi = 300)
plt.show()